In [2]:
# import shutil
# import os
# import zipfile
# # shutil.unpack_archive("gdrive/My Drive/train_dataset.tar.gz", "/content/gdrive/My Drive/FYP_dataset")

# os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [3]:
# !kaggle datasets download -d shahrukhkhan/im2latex100k

In [4]:
# zip_ref = zipfile.ZipFile('/content/im2latex100k.zip', 'r') #Opens the zip file in read mode
# zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
# zip_ref.close()

In [5]:
# len(os.listdir('/tmp/formula_images_processed/formula_images_processed/'))

In [2]:
!pip install jiwer
!pip install wandb
!pip install distance
# !pip install pytorch_lightning

In [3]:
import pandas as pd
import torch
from torch.utils.checkpoint import checkpoint
from torch.utils.data import DataLoader, Dataset
from torch import nn, Tensor
# import torchdata.datapipes as dp
import matplotlib.pyplot as plt
from pathlib import Path
import torchvision
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence
import glob
from tqdm.notebook import tqdm
import time
from jiwer import wer as cal_wer
# from nltk.metrics import edit_distance
from typing import Tuple
import pytorch_lightning as pl
import json
import random

In [6]:
bs = 2
epochs = 20
max_length = 150
log_idx = 300
workers = 2

cuda = torch.cuda.is_available()  
device = torch.device('cuda' if cuda else 'cpu')
# device = 'cpu'
device

device(type='cuda')

In [16]:
import re
class Text:
    def __init__(self):
        self.tokens = json.load(open("../input/latex-token/latex_tokens.json", "r"))
        self.map_tokens = dict(zip(self.tokens, range(len(self.tokens))))
        self.pad_id = 0
        self.sos_id = 1
        self.eos_id = 2
        self.TOKENIZE_PATTERN = re.compile(
            "(\\\\[a-zA-Z]+)|"
            +  # \[command name]
            # "(\{\w+?\})|"+ # {[text-here]} Check if this is needed
            '((\\\\)*[$-/:-?{-~!"^_`\[\]])|'
            + "(\w)|"  # math symbols
            + "(\\\\)"  # single letters or other chars
        )  # \ characters

    def tokenize(self, formula: str):
        """Returns list of tokens in given formula.
        formula - string containing the LaTeX formula to be tokenized
        Note: Somewhat work-in-progress"""
        tokens = re.finditer(self.TOKENIZE_PATTERN, str(formula))
        tokens = list(map(lambda x: x.group(0), tokens))
        tokens = [x for x in tokens if x is not None and x != ""]
        return tokens

    def int2text(self, x: int):
        if  x > self.eos_id :
            
             return self.tokens[x] 

    def int2text_n(self, x: Tensor):
        return "".join([self.tokens[i] for i in x if i > self.eos_id])

    def text2int(self, formula: str):
        return torch.LongTensor([self.map_tokens[i] for i in self.tokenize(formula)])

In [17]:
text = Text()
n_class = len(text.tokens)
n_class
520

520

In [14]:
from torch.utils.data import DataLoader, Dataset
data_path = Path('../input/file-30')
img_path = Path('../input/im2latex100k/formula_images_processed/formula_images_processed')
class LatexDataset(Dataset):
    def __init__(self, data_type: str):
        super().__init__()
        assert data_type in ['train', 'test', 'val'], 'Not found data type'
        csv_path = data_path / f'df_{data_type}_new.csv'
        df = pd.read_csv(csv_path)
        df['image'] = df.image.map(lambda x: img_path / x)
        self.walker = df.to_dict('records')
        
    def __len__(self):
        return len(self.walker)
    
    def __getitem__(self, idx):
        item = self.walker[idx]
        
        formula = item['formula']
        image = torchvision.io.read_image(str(item['image']))
        
        return image, formula

In [21]:
train_set = LatexDataset('train')
val_set = LatexDataset('val')
test_set = LatexDataset('test')

len(train_set), len(val_set), len(test_set)

(30000, 3000, 5000)

In [25]:
def collate_fn(batch):
    formulas = [text.text2int(i[1]) for i in batch]
    formulas = pad_sequence(formulas, batch_first=True)
    sos = torch.zeros(bs, 1) + text.map_tokens['<s>']
    eos = torch.zeros(bs, 1) + text.map_tokens['<e>']
    try:
        formulas = torch.cat((sos, formulas, eos), dim=-1).to(dtype=torch.long)
    except Exception:
        pass
        
    image = [i[0] for i in batch]
    width, height = 160, 200
    for img in image:
        c, h, w = img.size()
        # max_width = max(max_width, w)
        # max_height = max(max_height, h)
    pad = torchvision.transforms.Resize(size=(height, width))
    image = torch.stack(list(map(lambda x: pad(x), image))).to(dtype=torch.float)
    return image, formulas


class DataModule(pl.LightningDataModule):
    def __init__(self, train_set, val_set, test_set):
        super().__init__()
        self.train_set = train_set
        self.val_set = val_set
        self.test_set = test_set

    def train_dataloader(self):
        return DataLoader(self.train_set,
                          shuffle=True, 
                          batch_size=bs,
                          num_workers=workers,
                          collate_fn=collate_fn)
    
    def val_dataloader(self):
        return DataLoader(self.val_set,
                          shuffle=False, 
                          batch_size=bs,
                          num_workers=workers,
                          collate_fn=collate_fn)
    
    def test_dataloader(self):
        return DataLoader(self.test_set,
                          shuffle=False, 
                          batch_size=bs,
                          num_workers=workers,
                          collate_fn=collate_fn)

In [26]:
dm = DataModule(train_set, val_set, test_set)

In [27]:
import torch
import math

def add_positional_features(tensor: torch.Tensor,
                            min_timescale: float = 1.0,
                            max_timescale: float = 1.0e4):
    """
    Implements the frequency-based positional encoding described
    in `Attention is all you Need
    Parameters
    ----------
    tensor : ``torch.Tensor``
        a Tensor with shape (batch_size, timesteps, hidden_dim).
    min_timescale : ``float``, optional (default = 1.0)
        The largest timescale to use.
    Returns
    -------
    The input tensor augmented with the sinusoidal frequencies.
    """
    _, timesteps, hidden_dim = tensor.size()

    timestep_range = get_range_vector(timesteps, tensor.device).data.float()
    # We're generating both cos and sin frequencies,
    # so half for each.
    num_timescales = hidden_dim // 2
    timescale_range = get_range_vector(
        num_timescales, tensor.device).data.float()

    log_timescale_increments = math.log(
        float(max_timescale) / float(min_timescale)) / float(num_timescales - 1)
    inverse_timescales = min_timescale * \
        torch.exp(timescale_range * -log_timescale_increments)

    # Broadcasted multiplication - shape (timesteps, num_timescales)
    scaled_time = timestep_range.unsqueeze(1) * inverse_timescales.unsqueeze(0)
    # shape (timesteps, 2 * num_timescales)
    sinusoids = torch.randn(
        scaled_time.size(0), 2*scaled_time.size(1), device=tensor.device)
    sinusoids[:, ::2] = torch.sin(scaled_time)
    sinusoids[:, 1::2] = torch.sin(scaled_time)
    if hidden_dim % 2 != 0:
        # if the number of dimensions is odd, the cos and sin
        # timescales had size (hidden_dim - 1) / 2, so we need
        # to add a row of zeros to make up the difference.
        sinusoids = torch.cat(
            [sinusoids, sinusoids.new_zeros(timesteps, 1)], 1)
    return tensor + sinusoids.unsqueeze(0)


def get_range_vector(size: int, device) -> torch.Tensor:
    return torch.arange(0, size, dtype=torch.long, device=device)

In [28]:
import torchvision.models as models
class Encoder(nn.Module):
    def __init__(self, enc_out_dim: int, add_pos_feat = True):
        super(Encoder, self).__init__()
#         self.conv = nn.Sequential(
#             nn.Conv2d(3, 64, 3, 1, 1),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 1),
#             nn.BatchNorm2d(64),
            
#             nn.Conv2d(64, 128, 3, 1, 1),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 1),
#             nn.BatchNorm2d(128),
            
#             nn.Conv2d(128, 256, 3, 1, 1),
#             nn.ReLU(),
#             nn.BatchNorm2d(256),
            
#             nn.Conv2d(256, 256, 3, 1, 1),
#             nn.ReLU(),
#             nn.BatchNorm2d(256),
#             nn.Conv2d(256, enc_out_dim, 3, 1, 0),
#             nn.ReLU()
#         )
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 3, 1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 256, 3, 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, 1),
            nn.ReLU()
            nn.MaxPool2d(2, 1),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 512, 3, 1),
            nn.ReLU()
            nn.MaxPool2d(1, 2),
#             nn.BatchNorm2d(512),
            nn.Conv2d(512, enc_out_dim, 3, 1, 0),
            nn.ReLU()

        )
        self.add_pos_feat = add_pos_feat
    

    def forward(self, X: Tensor):
        """
            x: (bs, c, w, h)
        """
        features = self.conv(X)
        features = features.permute(0, 2, 3, 1)
#         features = features.view(features.size(0), -1, features.size(-1)) 
        B, H, W, _ = features.shape
        features = features.contiguous().view(B, H*W, -1)
        
        if self.add_pos_feat :
            features = add_positional_features(features)
        return features

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch.distributions.uniform import Uniform

In [30]:
INIT = 1e-2
class Decoder(nn.Module):
    def __init__(self, out_size, emb_size, dec_rnn_h, enc_out_dim, n_layer = 1, dropout = 0., beam_width = 5, text = text):
        super(Decoder, self).__init__()
        self.rnn_decoder = nn.LSTMCell(dec_rnn_h + emb_size, dec_rnn_h)
        self.embedding = nn.Embedding(out_size, emb_size)
        
        self.init_wh = nn.Linear(enc_out_dim, dec_rnn_h)
        self.init_wc = nn.Linear(enc_out_dim, dec_rnn_h)
        self.init_wo = nn.Linear(enc_out_dim, dec_rnn_h)
        
        self.beta = nn.Parameter(torch.Tensor(enc_out_dim))
        init.uniform_(self.beta, -INIT, INIT)
        self.W_1 = nn.Linear(enc_out_dim, enc_out_dim, bias = False)
        self.W_2 = nn.Linear(dec_rnn_h, enc_out_dim, bias = False)
        
        self.W_3 = nn.Linear(dec_rnn_h + enc_out_dim, dec_rnn_h, bias = False)
        self.W_out = nn.Linear(dec_rnn_h, out_size, bias = False)
        
        self.dropout = nn.Dropout(p= dropout)
        self.uniform  = Uniform(0, 1)
        self.beam_width = beam_width
        self.logsoftmax = nn.LogSoftmax(dim=-1)
        self.text  = text
        
        
    def forward(self, features, formulas, epsilon=1.):
        """args:
        imgs: [B, C, H, W]
        formulas: [B, MAX_LEN]
        epsilon: probability of the current time step to
                 use the true previous token
        return:
        logits: [B, MAX_LEN, VOCAB_SIZE]
        """
        # encoding
        # init decoder's states
        dec_states, o_t = self.init_decoder(features)
        max_len = formulas.size(1)
        logits = []
        for t in range(max_len):
            tgt = formulas[:, t:t+1]
            # schedule sampling
#             if logits and self.uniform.sample().item() > epsilon:
#                 tgt = torch.argmax(torch.log(logits[-1]), dim=1, keepdim=True)
            # ont step decoding
            dec_states, o_t, logit = self.decode_step(
                dec_states, o_t, features, tgt)
            logits.append(logit)
        logits = torch.stack(logits, dim=1)  # [B, MAX_LEN, out_size]
        return logits
        
        
    def decode_step(self, dec_states, o_t, enc_out, tgt):
        """Runing one step decoding"""

        prev_y = self.embedding(tgt).squeeze(1)  # [B, emb_size]
        inp = torch.cat([prev_y, o_t], dim=1)  # [B, emb_size+dec_rnn_h]
        h_t, c_t = self.rnn_decoder(inp, dec_states)  # h_t:[B, dec_rnn_h]
        h_t = self.dropout(h_t)
        c_t = self.dropout(c_t)

        # context_t : [B, C]
        context_t, attn_scores = self._get_attn(enc_out, h_t)

        # [B, dec_rnn_h]
        o_t = self.W_3(torch.cat([h_t, context_t], dim=1)).tanh()
        o_t = self.dropout(o_t)

        # calculate logit
        logit = self.logsoftmax(self.W_out(o_t))  # [B, out_size]

        return (h_t, c_t), o_t, logit
    
    def _get_attn(self, enc_out, h_t):
        """Attention mechanism
        args:
            enc_out: row encoder's output [B, L=H*W, C]
            h_t: the current time step hidden state [B, dec_rnn_h]
        return:
            context: this time step context [B, C]
            attn_scores: Attention scores
        """
        # cal alpha
        alpha = torch.tanh(self.W_1(enc_out)+self.W_2(h_t).unsqueeze(1))
        alpha = torch.sum(self.beta*alpha, dim=-1)  # [B, L]
        alpha = F.softmax(alpha, dim=-1)  # [B, L]

        # cal context: [B, C]
        context = torch.bmm(alpha.unsqueeze(1), enc_out)
        context = context.squeeze(1)
        return context, alpha
                       
    def generate_formulas(self, features, max_len, type):
        assert type in ['greedy', 'beam']
        if type == 'greedy':
            dec_states, O_t = self.init_decoder(features)
            batch_size = features.size(0)
            tgt = torch.ones(batch_size, 1, device = device).long() * self.text.sos_id
#             formulas_idx = torch.ones(batch_size, max_len, device=device).long() * self.text.pad_id
            result = []
            for t in range(max_len):
                dec_states, O_t, logit = self.decode_step(dec_states, O_t, features, tgt)
                
                tgt = torch.argmax(logit, dim=1)
                result.append(tgt.item())
            return result
        
        if type == 'beam':
            dec_states, O_t = self.init_decoder(features)
            batch_size = features.size(0)
            list_candidates = [([self.text.sos_id], dec_states, O_t, 0.0)]
            for t in range(max_length):
                new_candidates = []
                for inp, dec_states, O_t, log_prob in list_candidates:
                       tgt = torch.ones(batch_size, 1, device = device).long() * inp[-1]
                       dec_states, O_t, logit = self.decode_step(dec_states, O_t, features, tgt)
                       topk = logit.topk(self.beam_width)
                       new_log_prob = topk.values.view(-1).tolist()
                       new_idx = topk.indices.view(-1).tolist()
                       for val, idx in zip(new_log_prob, new_idx):
                            new_inp = inp + [idx]
                            new_candidates.append((new_inp, dec_states, O_t, log_prob + val))
                new_candidates = sorted(new_candidates, key = lambda x: x[3], reverse=True)
                list_candidate = new_candidates[:self.beam_width]
            return list_candidate[0][0]
    

    def _init_h(self, mean_enc_out):
        return torch.tanh(self.init_wh(mean_enc_out))
    
    
    
    def _init_c(self, mean_enc_out):
        return torch.tanh(self.init_wc(mean_enc_out))
    
    def _init_o(self, mean_enc_out):
        return torch.tanh(self.init_wo(mean_enc_out))
    
    def init_decoder(self, enc_out):
        '''
        args:
        enc_out : the output of row encoder [B, H*W, C]
        return:
        h_0, c_0 : shape [B, dec_rnn_h]
        init_o : the average of enc_out [B, dec_rnn_h]
        '''
        mean_enc_out = enc_out.mean(dim=1)
        h = self._init_h(mean_enc_out)
        c = self._init_c(mean_enc_out)
        init_o = self._init_o(mean_enc_out)
        return (h, c), init_o
        
        

In [31]:
class EncoderDecoder(nn.Module):
    def __init__(self, enc_out_dim, out_size, emb_size, dec_rnn_h, text):
        super().__init__()
        self.encoder = Encoder(enc_out_dim = enc_out_dim)
        self.decoder = Decoder(out_size = out_size, dec_rnn_h = dec_rnn_h, emb_size = emb_size, enc_out_dim= enc_out_dim, text= text)
    def forward(self, images,formulas):
        features = self.encoder(images)
        outputs = self.decoder(features, formulas)
        return outputs

In [32]:
import nltk
import distance
import numpy as np

def edit_distance(references, hypotheses):
    """Computes Levenshtein distance between two sequences.
    Args:
        references: list of list of token (one hypothesis)
        hypotheses: list of list of token (one hypothesis)
    Returns:
        1 - levenshtein distance: (higher is better, 1 is perfect)
    """
    d_leven, len_tot = 0, 0
    for ref, hypo in zip(references, hypotheses):
        d_leven += distance.levenshtein(ref, hypo)
        len_tot += float(max(len(ref), len(hypo)))

    return 1. - d_leven / len_tot

def calculate_bleu_score(references, hypotheses):
    """Computes bleu score.
    Args:
        references: list of list (one hypothesis)
        hypotheses: list of list (one hypothesis)
    Returns:
        BLEU-4 score: (float)
    """
    references = [[ref] for ref in references]  # for corpus_bleu func
    BLEU_4 = nltk.translate.bleu_score.corpus_bleu(
        references, hypotheses,
        weights=(0.25, 0.25, 0.25, 0.25)
    )
    return BLEU_4

def exact_match_score(references, hypotheses):
    """Computes exact match scores.
    Args:
        references: list of list of tokens (one ref)
        hypotheses: list of list of tokens (one hypothesis)
    Returns:
        exact_match: (float) 1 is perfect
    """
    exact_match = 0
    for ref, hypo in zip(references, hypotheses):
        if np.array_equal(ref, hypo):
            exact_match += 1

    return exact_match / float(max(len(hypotheses), 1))

In [45]:
from torchtext.data.metrics import bleu_score
class Image2LatexModel(pl.LightningModule):
    def __init__(self, init_lr, max_lr, total_steps, **kwargs):
        super().__init__()
        self.model = EncoderDecoder(**kwargs)
        self.criterion = nn.CrossEntropyLoss()
        self.init_lr = init_lr
        self.max_lr = max_lr
        self.total_steps = total_steps
        self.save_hyperparameters()
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr= self.init_lr)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr= self.max_lr, total_steps= self.total_steps)
        return [optimizer], [scheduler]
    def forward(self, images, formulas):
        return self.model(images, formulas)

    def training_step(self, batch, batch_idx):
        images, formulas = batch
        targets = formulas[:,1:]
        formulas_in = formulas[:,:-1]
        outputs = self.model(images, formulas_in)
        bs, t, _ = outputs.size()
        outputs = outputs.reshape(bs * t, -1)

        
        loss = self.criterion(outputs, targets.reshape(-1))
        self.log("train loss", loss, on_step=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        images, formulas = batch
        targets = formulas[:,1:]
        formulas_in = formulas[:,:-1]
        outputs = self.model(images, formulas_in)
        bs, t, _ = outputs.size()
        outputs = outputs.reshape(bs * t, -1)

        
        loss = self.criterion(outputs, targets.reshape(-1))

        predicts = [text.tokenize(text.int2text_n(self.model.decoder.generate_formulas(self.model.encoder(i.unsqueeze(0))
                                                                                       , max_len=150, type = 'greedy'))) for i in images]

        truths = [text.tokenize(text.int2text_n(i)) for i in formulas]

        edit_dist = edit_distance(predicts, truths)
        bleu =  calculate_bleu_score(predicts, truths)
        em = exact_match_score(predicts, truths)
        
        self.log('val  loss', loss, on_step= True)
        self.log('val edit_dist', edit_dist, on_step= True)
        self.log('val bleu score', bleu, on_step= True)
        self.log('exact math', em, on_step= True)

    
        return loss
    
    def test_step(self, batch, batch_idx):
        images, formulas = batch
        targets = formulas[:,1:]
        formulas_in = formulas[:,:-1]
        outputs = self.model(images, formulas_in)
        bs, t, _ = outputs.size()
        outputs = outputs.reshape(bs * t, -1)

        
        loss = self.criterion(outputs, targets.reshape(-1))
        
        predicts = [text.tokenize(text.int2text_n(self.model.decoder.generate_formulas(self.model.encoder(i.unsqueeze(0)),
                                                                                       max_len=150, type = 'greedy'))) for i in images]

        truths = [text.tokenize(text.int2text_n(i)) for i in formulas]

        edit_dist = edit_distance(predicts, truths)
        bleu =  calculate_bleu_score(predicts, truths)
        em = exact_match_score(predicts, truths)
        
        self.log('test  loss', loss)
        self.log('test edit_dist', edit_dist)
        self.log('test bleu score', bleu)
#         self.log('exact math', em)

    
        return loss
        

In [ ]:
# 5adc273c49fc5b2641ffbfcce7c24338b108c7cb

In [34]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [35]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer

wandb_logger = WandbLogger(project="im2lax_paper")

wandb: Currently logged in as: philongds145. Use `wandb login --relogin` to force relogin


In [46]:
embed_size=80 
vocab_size = n_class
encoder_dim=512
decoder_dim=256
init_lr = 1e-4
max_lr = 1e-3
total_steps = epochs * bs
model = Image2LatexModel(init_lr= init_lr, max_lr= max_lr, total_steps= total_steps,
                         enc_out_dim = encoder_dim, out_size = vocab_size,
                         emb_size= embed_size, dec_rnn_h = decoder_dim , text = text)

In [47]:
checkpoint_callback  = pl.callbacks.ModelCheckpoint(dirpath='./', save_on_train_epoch_end=True)
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
trainer = pl.Trainer(logger=wandb_logger, callbacks=[ checkpoint_callback, lr_monitor],
                     max_epochs=epochs, accelerator='gpu')
trainer.test(datamodule=dm, model=model, ckpt_path='../input/ckpt-5/epoch5bn.ckpt')

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test  loss         │    0.6824802160263062     │
│      test bleu score      │    0.5059991478919983     │
│      test edit_dist       │    0.5914739966392517     │
└───────────────────────────┴───────────────────────────┘

[{'test  loss': 0.6824802160263062,
  'test edit_dist': 0.5914739966392517,
  'test bleu score': 0.5059991478919983}]